# Implementation of face recognition using neural net 

In [1]:
%matplotlib inline

In [2]:
import cv2
import numpy as np
import os
from skimage import io
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

In [3]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

Using Theano backend.


#### Listing the path of all the images

In [4]:
DatasetPath = []
for i in os.listdir("yalefaces"):
    DatasetPath.append(os.path.join("yalefaces", i))

#### Reading each image and assigning respective labels

In [5]:
imageData = []
imageLabels = []

In [6]:
for i in DatasetPath:
    imgRead = io.imread(i,as_grey=True)
    imageData.append(imgRead)
    
    labelRead = int(os.path.split(i)[1].split(".")[0].replace("subject", "")) - 1
    imageLabels.append(labelRead)

#### Preprocessing: Face Detection using OpenCV and cropping the image to a size of 150 * 150

In [7]:
faceDetectClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [8]:
imageDataFin = []
for i in imageData:
    facePoints = faceDetectClassifier.detectMultiScale(i)
    x,y = facePoints[0][:2]
    cropped = i[y: y + 150, x: x + 150]
    imageDataFin.append(cropped)

In [9]:
c = np.array(imageDataFin)

In [10]:
c.shape

(165, 150, 150)

#### Splitting Dataset into train and test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(np.array(imageDataFin),np.array(imageLabels), train_size=0.9, random_state = 20)

In [12]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [13]:
X_train.shape

(148, 150, 150)

In [14]:
X_test.shape

(17, 150, 150)

In [15]:
nb_classes = 15
y_train = np.array(y_train) 
y_test = np.array(y_test) 

In [16]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

#### Converting each 2d image into 1D vector

In [17]:
X_train = X_train.reshape(148, 150*150)
X_test = X_test.reshape(17, 150*150)

In [18]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [19]:
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Training matrix shape (148, 22500)
Testing matrix shape (17, 22500)


#### Defining the model

In [20]:
model = Sequential()
model.add(Dense(512,input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [21]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 512)           11520512    dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 512)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 512)           0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 512)           262656      dropout_1[0][0]                  
___________________________________________________________________________________________

In [22]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

#### Training

In [23]:
model.fit(X_train, Y_train, batch_size=64, nb_epoch=50, verbose=1, validation_data=(X_test, Y_test))

Train on 148 samples, validate on 17 samples
Epoch 1/50
148/148 [==============================] - 0s - loss: 7.5383 - acc: 0.0676 - val_loss: 8.6825 - val_acc: 0.1176
Epoch 2/50
148/148 [==============================] - 0s - loss: 10.7414 - acc: 0.1486 - val_loss: 6.8926 - val_acc: 0.1176
Epoch 3/50
148/148 [==============================] - 0s - loss: 10.0452 - acc: 0.1959 - val_loss: 8.6226 - val_acc: 0.2941
Epoch 4/50
148/148 [==============================] - 0s - loss: 9.6777 - acc: 0.2568 - val_loss: 8.7041 - val_acc: 0.4118
Epoch 5/50
148/148 [==============================] - 0s - loss: 9.2544 - acc: 0.3176 - val_loss: 8.6605 - val_acc: 0.2941
Epoch 6/50
148/148 [==============================] - 0s - loss: 8.3280 - acc: 0.3716 - val_loss: 8.4451 - val_acc: 0.4118
Epoch 7/50
148/148 [==============================] - 0s - loss: 8.4121 - acc: 0.3514 - val_loss: 8.4700 - val_acc: 0.3529
Epoch 8/50
148/148 [==============================] - 0s - loss: 7.7824 - acc: 0.4189 - val_

#### Evaluating the performance

In [24]:
loss, accuracy = model.evaluate(X_test,Y_test, verbose=0)

In [25]:
loss

0.24348054826259613

In [26]:
accuracy

0.88235294818878174

In [27]:
predicted_classes = model.predict_classes(X_test)

17/17 [==============================] - 0s


In [28]:
correct_classified_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_classified_indices = np.nonzero(predicted_classes != y_test)[0]

In [29]:
correct_classified_indices

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16])

In [30]:
incorrect_classified_indices

array([ 0, 13])